## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 
Информация о таблице

In [1]:
import pandas as pd
import numpy as np
import pprint

from pymystem3 import Mystem
from collections import Counter

df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Посмотрим на первые 10 строк


In [2]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Подсчитаем пустые значения

In [3]:
print(df.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Подсчитаем пропущенные значения


In [4]:
print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Поищем данные с отклонениями

In [5]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Опишем значения столбцов (для удобства, что бы не возвращаться в меню)  
children — количество детей в семье  
days_employed — общий трудовой стаж в днях  
dob_years — возраст клиента в годах  
education — уровень образования клиента  
education_id — идентификатор уровня образования  
family_status — семейное положение  
family_status_id — идентификатор семейного положения  
gender — пол клиента  
income_type — тип занятости  
debt — имел ли задолженность по возврату кредитов  
total_income — ежемесячный доход  
purpose — цель получения кредита  

### Вывод

На первый взгляд видим следующие проблемы, которые могут  затруднить анализ данных:  
      1.отрицательные значения в  колонках children и days_employed  
      2.странно одинаковое количество пропущенных данных в столбцах days_employed и total_income -рассмотреть внимательнее, может есть закономерность  
      3.минимальный возраст указан "0" - определить много ли таких строк  
      4.написание значений в столбце education - привести к единому виду  
      5.указание стажа в днях не очень удобно для восприятия  

### Шаг 2. Предобработка данных

### Обработка пропусков

Для начала заполним пропуски нулями, считаю это самым разумным в данном случае т.к. стаж и ежемесячный доход являются существенными данными влияющими на получение и размер кредита и заполнять их средними значениями или какими-то еще, значит ввести в заблужение систему выдачи кредитов.

In [6]:
#заполняем пропуски нулевыми значениями в столбце 'days_employed'
df['days_employed'] = df['days_employed'].fillna(0)

In [7]:
#заполняем пропуски нулевыми значениями в столбце 'total_income'
df['total_income'] = df['total_income'].fillna(0)

Проверим что получилось

In [8]:
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Взяла самый простой способ который существенно не повлияет на свойство данных, по хорошему необходимо выделить кусок с проблемными данными и запросить дороботку у предоставивших данные.

### Замена типа данных

Самое простое: Приведем строки в education к нижнему регистру, не влияет на решение о получении кредита, но не красиво))

In [9]:
df['education'] = df['education'].str.lower()

In [10]:
#проверим
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Еще простое -количество детей, как мы уже видели, минимальное количество -1 и максимальное 20

In [11]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Выясним сколько у нас трок со странными значениями -1 и 20

In [12]:
df[df['children'] == -1]['children'].count()

47

In [13]:
df[df['children'] == 20]['children'].count()

76

С отрицательным значением, более или менее, все понятно просто уберем "-"

In [14]:
df['children'] = df['children'].replace(-1, 1)

In [15]:
#проверим
df[df['children'] == -1]['children'].count()

0

Что делать с данными о 20 детях? Есть вероятность, что эти данные верны (семейный детский дом и что-то такое). Также в реальности можно запросить уточнение у автора данных. В нашем случае, и из-за незначительного количества, считаю стот просто свести данные к простым и понятным, т.е. убрать "0".

In [16]:
df['children'] = df['children'].replace(20, 2)

In [17]:
#проверим
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

Еще одна аномалия это возраст, минимальный 0 лет. Кредиты выдаются людям от 18 лет, посмотрим сколько таких строк. 

In [18]:
df[df['dob_years'] < 18]['dob_years'].count()

101

In [19]:
#пол
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Внезапно пола больше чем два, посомтрим сколько у нас таинственного 'XNA'

In [20]:
df[df['gender'] == 'XNA']['gender'].count()

1

In [21]:
#статус
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [22]:
#семейное положение
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [23]:
#ежемесячный доход приведем в более читаемый вид
df['total_income'] = df['total_income'].astype(int)

In [24]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


 Осталось самое проблемное и непонятное  days_employed (рабочий стаж)

In [25]:
df['days_employed'].describe()

count     21525.000000
mean      56678.874622
std      134870.763085
min      -18388.949901
25%       -2518.168900
50%        -982.531720
75%           0.000000
max      401755.400475
Name: days_employed, dtype: float64

In [26]:
# переводим отрицательные значения в положительные
df['days_employed'] = df['days_employed'].abs()

In [27]:
#проверим
df['days_employed'].describe()

count     21525.000000
mean      60156.419005
std      133355.929525
min           0.000000
25%         610.652074
50%        1808.053434
75%        4779.587738
max      401755.400475
Name: days_employed, dtype: float64

Максимальный стаж 401755 дней, это 1100лет, необъяснимая и не реальная цифра. Ради любопытства хочу посмотреть возраст людей с таким стажем.

In [28]:
df[df['days_employed'] > 400000].groupby('dob_years')['dob_years'].count()

dob_years
0     1
42    1
46    1
47    1
49    3
50    2
52    4
53    2
54    2
55    4
56    7
57    9
58    5
59    3
60    6
61    7
62    4
63    4
64    3
65    1
66    2
67    5
68    2
69    2
70    1
72    1
Name: dob_years, dtype: int64

Возможно есть какие-то закономерности между этими аретфактами и другой информацией, но т.к. нет возможности узнать подробности получения этих данных, не вижу смысла терять на это время. 

### Вывод

Обработали данные и выявили проблемные места.  
 1. Обнаружена 101 запись о людях с возрастом до 18 лет, надо будет иметь это ввиду при дальнейшей анализе.  
 2. 1 запись со странным гендером 'XNA' считаю можно игнорировать, т.к. количество незначительное и пол не влияет на получение кредита.  
 3. Самые странные данные это записи о стаже с нереальными сроками, но я решила их игнорировать т.к. нет полностью достоверного способа исправить их.

### Обработка дубликатов

In [29]:
#ищем дубли
df.duplicated().sum()

71

In [30]:
#удаляем
df = df.drop_duplicates().reset_index(drop=True)

In [31]:
#проверяем
df.duplicated().sum()

0

### Вывод

Счтаю правильным удалять дубли после всех обработок, исключает возможность что-то забыть.

### Лемматизация

In [32]:
#Узнаем количество значений в нужном столбце
purpose_unique = df['purpose'].unique().tolist()
purpose_unique

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

In [33]:
m = Mystem()
text = '; '.join(purpose_unique)
pprint.pprint(text)

('покупка жилья; приобретение автомобиля; дополнительное образование; сыграть '
 'свадьбу; операции с жильем; образование; на проведение свадьбы; покупка '
 'жилья для семьи; покупка недвижимости; покупка коммерческой недвижимости; '
 'покупка жилой недвижимости; строительство собственной недвижимости; '
 'недвижимость; строительство недвижимости; на покупку подержанного '
 'автомобиля; на покупку своего автомобиля; операции с коммерческой '
 'недвижимостью; строительство жилой недвижимости; жилье; операции со своей '
 'недвижимостью; автомобили; заняться образованием; сделка с подержанным '
 'автомобилем; получение образования; автомобиль; свадьба; получение '
 'дополнительного образования; покупка своего жилья; операции с недвижимостью; '
 'получение высшего образования; свой автомобиль; сделка с автомобилем; '
 'профильное образование; высшее образование; покупка жилья для сдачи; на '
 'покупку автомобиля; ремонт жилью; заняться высшим образованием')


In [34]:
lemmas = m.lemmatize(text)
pprint.pprint(Counter(lemmas))

Counter({' ': 59,
         '; ': 37,
         'покупка': 10,
         'недвижимость': 10,
         'автомобиль': 9,
         'образование': 9,
         'жилье': 7,
         'с': 5,
         'операция': 4,
         'на': 4,
         'свой': 4,
         'свадьба': 3,
         'строительство': 3,
         'получение': 3,
         'высокий': 3,
         'дополнительный': 2,
         'для': 2,
         'коммерческий': 2,
         'жилой': 2,
         'подержать': 2,
         'заниматься': 2,
         'сделка': 2,
         'приобретение': 1,
         'сыграть': 1,
         'проведение': 1,
         'семья': 1,
         'собственный': 1,
         'со': 1,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})


Итог, главные цели кредитов: автомобиль, недвижимость, образование, свадьба

In [35]:
#сделаем словарь
purpose_keys= []
purpose_unique = list(df.purpose.unique())
for value in range(len(purpose_unique)):
    if 'образован' in purpose_unique[value]:
        purpose_keys.append('Образование')
    elif 'авто' in purpose_unique[value]:
        purpose_keys.append('Автомобиль')
    elif 'свадь' in purpose_unique[value]:
        purpose_keys.append('Свадьба')
    else:
        purpose_keys.append('Недвижимость')
        
purpose_keys

['Недвижимость',
 'Автомобиль',
 'Образование',
 'Свадьба',
 'Недвижимость',
 'Образование',
 'Свадьба',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Автомобиль',
 'Автомобиль',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Автомобиль',
 'Образование',
 'Автомобиль',
 'Образование',
 'Автомобиль',
 'Свадьба',
 'Образование',
 'Недвижимость',
 'Недвижимость',
 'Образование',
 'Автомобиль',
 'Автомобиль',
 'Образование',
 'Образование',
 'Недвижимость',
 'Автомобиль',
 'Недвижимость',
 'Образование']

In [36]:
purpose_unique

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

In [37]:
#свяжем эти два элемента и посмотрим на результат
purpose_list = dict(zip(purpose_unique, purpose_keys))
purpose_list

{'покупка жилья': 'Недвижимость',
 'приобретение автомобиля': 'Автомобиль',
 'дополнительное образование': 'Образование',
 'сыграть свадьбу': 'Свадьба',
 'операции с жильем': 'Недвижимость',
 'образование': 'Образование',
 'на проведение свадьбы': 'Свадьба',
 'покупка жилья для семьи': 'Недвижимость',
 'покупка недвижимости': 'Недвижимость',
 'покупка коммерческой недвижимости': 'Недвижимость',
 'покупка жилой недвижимости': 'Недвижимость',
 'строительство собственной недвижимости': 'Недвижимость',
 'недвижимость': 'Недвижимость',
 'строительство недвижимости': 'Недвижимость',
 'на покупку подержанного автомобиля': 'Автомобиль',
 'на покупку своего автомобиля': 'Автомобиль',
 'операции с коммерческой недвижимостью': 'Недвижимость',
 'строительство жилой недвижимости': 'Недвижимость',
 'жилье': 'Недвижимость',
 'операции со своей недвижимостью': 'Недвижимость',
 'автомобили': 'Автомобиль',
 'заняться образованием': 'Образование',
 'сделка с подержанным автомобилем': 'Автомобиль',
 'полу

In [38]:
# новый столбец, будет удобненько для выводов
df['purpose_dict'] = df['purpose'].map(purpose_list)

In [39]:
#проверим. удобно вместо громадного списка всего 4 пункта к тоторым можно привязать что угодно
df['purpose_dict'].unique()

array(['Недвижимость', 'Автомобиль', 'Образование', 'Свадьба'],
      dtype=object)

In [40]:
df.groupby('purpose_dict')['debt'].count()

purpose_dict
Автомобиль       4306
Недвижимость    10811
Образование      4013
Свадьба          2324
Name: debt, dtype: int64

### Категоризация данных

Посмотрим на какие цели берут и не возвращают большую часть кредитов


In [41]:
purpose_debt = df.pivot_table(index=['purpose_dict'], columns=['debt'], values='children', aggfunc='count')
purpose_debt['ratio'] = purpose_debt[1] / purpose_debt[0] * 100
purpose_debt

debt,0,1,ratio
purpose_dict,,,
Автомобиль,3903,403,10.325391
Недвижимость,10029,782,7.797388
Образование,3643,370,10.156464
Свадьба,2138,186,8.699719


Посмотрим как невозврат кредитов  зависит от количества детей


In [42]:
children_debt = df.pivot_table(index = ['children'], columns = ['debt'], values = 'purpose_dict', aggfunc='count')
children_debt['ratio'] = children_debt[1] / children_debt[0] * 100
children_debt

debt,0,1,ratio
children,,,
0,13028.0,1063.0,8.159349
1,4410.0,445.0,10.090703
2,1926.0,202.0,10.488058
3,303.0,27.0,8.910891
4,37.0,4.0,10.810811
5,9.0,NaN,NaN


Посмотрим как невозврат кредитов  зависит от семейного положения


In [43]:
family_status_debt = df.pivot_table(index = ['family_status'], columns = ['debt'], values = 'purpose_dict', aggfunc='count')
family_status_debt['ratio'] = family_status_debt[1] / family_status_debt[0] * 100
family_status_debt

debt,0,1,ratio
family_status,,,
Не женат / не замужем,2536,274,10.804416
в разводе,1110,85,7.657658
вдовец / вдова,896,63,7.031250
гражданский брак,3763,388,10.310922
женат / замужем,11408,931,8.160940


Чуть сложнее с определить зависимость от материального положения. разделим людей в группы по уровню дохода


In [44]:
def income_status(total_income):
    if total_income <= 35000:
            return 'Низкий'
    if total_income <= 100000:
            return 'Средний'
    if total_income < 300000:
            return 'Высокий'
    return 'Очень высокий'

In [45]:
df['income_status'] = df['total_income'].apply(income_status)

In [46]:
#проверим
df['income_status'].unique()

array(['Высокий', 'Средний', 'Низкий', 'Очень высокий'], dtype=object)

In [47]:
total_income_debt = df.pivot_table(index = ['income_status'], columns = ['debt'], values = 'purpose_dict', aggfunc='count')
total_income_debt['ratio'] = total_income_debt[1] / total_income_debt[0] * 100
total_income_debt

debt,0,1,ratio
income_status,,,
Высокий,12295,1111,9.036194
Низкий,1999,173,8.654327
Очень высокий,1376,106,7.703488
Средний,4043,351,8.681672


### Вывод

Все результаты выглядят логичто и объяснимо. Есть сомнения в результатах зависимости просрочки кредитов  от задолженности, но тут очень большой объем различных данных, возможно необходима более узкая классификация и привязка к другим показателям.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Предлагаю отбросить данные о 4-5 детей, т.к. это слишком маленький процент от общего числа подающих заявки на кредит. Из остальных данных видно что резкой зависимости в зависимости от количества детейй не видно. Возможно результаты будут более понятные если взять статистику по количеству людей с разным количеством детей подающих заявление на выдачу кредита.

In [48]:
df['children'].value_counts(normalize = True)

0    0.656801
1    0.226298
2    0.099189
3    0.015382
4    0.001911
5    0.000420
Name: children, dtype: float64

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Интересные показатели- люди живущие в паре чаще берут кредиты и реже их отдают. Даже если взять сумарное количество тех кто живут в паре и одиноких людей. Видимо, есть зависимость от наличия второго кормильца в семье и потребностей, парам легче решиться на кредит.

In [49]:
df['family_status'].value_counts(normalize = True)

женат / замужем          0.575138
гражданский брак         0.193484
Не женат / не замужем    0.130978
в разводе                0.055701
вдовец / вдова           0.044700
Name: family_status, dtype: float64

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Явная зависимость от низкого дохода, что логично и понятно. Тут, считаю необходимо брать более развернутую выборку, т.к. люди с высоким и очень высоким доходом, врядли берут кредиты на какие-то мелкие бытовые нужды. Также, возможно, есть зависимость от суммы кредита.

In [50]:
df['income_status'].value_counts(normalize = True)

Высокий          0.624872
Средний          0.204810
Низкий           0.101240
Очень высокий    0.069078
Name: income_status, dtype: float64

- Как разные цели кредита влияют на его возврат в срок?

Также как и в других случаях, необходимо понимать что кредит на вкартиру это самый популярный вид кредита от общего процента всех выдаваемых. 

In [51]:
df['purpose_dict'].value_counts(normalize = True)

Недвижимость    0.503915
Автомобиль      0.200708
Образование     0.187051
Свадьба         0.108325
Name: purpose_dict, dtype: float64

### Шаг 4. Общий вывод

На основе данного датасета нет точной возможности сделать полные выводы: даже после очистки и обработки данных, очень много вопросов к качеству.  
1.Данные о стаже работы явно нуждаются в выделении и дополнительных запросах т.к. на соответсвуют возрасту людей, имеют нереальные сроки и не понятно в каких величинах вычисляются;  
2.Есть некорректные данные о возрасте: есть сомнения что люди до 18-и лет подают заявки; есть строки где возраст равыен "0", это или халатность опереатора данных или ошибка обработки в банке;  
3.Вызывают сомнения и данны о количестве детей - проблемы те же что и в первых двух пунктах;

Как общий вывод: в идеале необходимо выбрать все данные которые имеют необъяснимые артефакты и отбросить их из анализа. 

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [ ]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [ ]  выделены леммы в значениях столбца с целями получения кредита;
- [ ]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [ ]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [ ]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.